# **Homework 8 - Anomaly Detection**

If there are any questions, please contact mlta-2023-spring@googlegroups.com

Slide:    [Link](https://docs.google.com/presentation/d/18LkR8qulwSbi3SVoLl1XNNGjQQ_qczs_35lrJWOmHCk/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/t/c76950cc460140eba30a576ca7668d28)

# Set up the environment


## Package installation

In [1]:
# # Training progress bar
# !pip install -q qqdm

## Downloading data

In [2]:
# !git clone https://github.com/chiyuanhsiao/ml2023spring-hw8

# Import packages

In [3]:
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch.optim import Adam, AdamW
from qqdm import qqdm, format_str
import pandas as pd

# Loading data

In [4]:
train = np.load('trainingset.npy', allow_pickle=True, encoding='ASCII')
test = np.load('testingset.npy', allow_pickle=True, encoding='ASCII')

print(train.shape)
print(test.shape)

(100000, 64, 64, 3)
(19636, 64, 64, 3)


## Random seed
Set the random seed to a certain value for reproducibility.

In [5]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(42)

# Autoencoder

# Models & loss

In [6]:
class fcn_autoencoder(nn.Module):
    def __init__(self):
        super(fcn_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(size*size * 3, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, size*size*3), 
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


class conv_autoencoder(nn.Module):
    def __init__(self):
        super(conv_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2, padding=1),      
            nn.LeakyReLU(0.1),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),        
            
        )

        self.fcn_encoder = nn.Sequential(
            nn.Linear(64 * 64 * 3, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
        )
        self.fcn = nn.Linear(128*16*16,32)
        self.fcn2 = nn.Linear(32,128*16*16)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),       
            nn.LeakyReLU(0.1),
            nn.ConvTranspose2d(64, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )
    def forward(self, x):
        x1 = self.fcn_encoder(x.reshape([x.shape[0],3*64*64]))
        x = self.encoder(x)
        x = x.reshape([x.shape[0],128*16*16])
        x = self.fcn(x)
        x += x1
        x = self.fcn2(x)
        x = x.reshape([x.shape[0],128,16,16])
        x = self.decoder(x)
        return x


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),    
            nn.ReLU(),
        )
        self.enc_out_1 = nn.Sequential(
            nn.Conv2d(24, 48, 4, stride=2, padding=1),  
            nn.ReLU(),
        )
        self.enc_out_2 = nn.Sequential(
            nn.Conv2d(24, 48, 4, stride=2, padding=1),
            nn.ReLU(),
        )
        # Hint: can add more layers to encoder and decoder
        self.decoder = nn.Sequential(
			      nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1), 
            nn.ReLU(),
			      nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1), 
            nn.Tanh(),
        )

    def encode(self, x):
        h1 = self.encoder(x)
        return self.enc_out_1(h1), self.enc_out_2(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


def loss_vae(recon_x, x, mu, logvar, criterion):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    mse = criterion(recon_x, x)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    return mse + KLD

# Dataset module

Module for obtaining and processing data. The transform function here normalizes image's pixels from [0, 255] to [-1.0, 1.0].


In [7]:
class CustomTensorDataset(TensorDataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors):
        self.tensors = tensors
        if tensors.shape[-1] == 3:
            self.tensors = tensors.permute(0, 3, 1, 2)
        
        self.transform = transforms.Compose([
          transforms.Lambda(lambda x: x.to(torch.float32)),
          transforms.Lambda(lambda x: 2. * x/255. - 1.),
            transforms.CenterCrop(size)
        ])
        
    def __getitem__(self, index):
        x = self.tensors[index]
        
        if self.transform:
            # mapping images to [-1.0, 1.0]
            x = self.transform(x)

        return x

    def __len__(self):
        return len(self.tensors)

# Training

## Configuration


In [8]:
# Training hyperparameters
num_epochs = 100
batch_size = 128 # Hint: batch size may be lower
learning_rate = 1e-3
size = 42
# Build training dataloader
x = torch.from_numpy(train)
train_dataset = CustomTensorDataset(x)

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Model
model_type = 'fcn'   # selecting a model type from {'cnn', 'fcn', 'vae', 'resnet'}
model_classes = {'fcn': fcn_autoencoder(), 'cnn': conv_autoencoder(), 'vae': VAE()}
model = model_classes[model_type].cuda()


# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


## Training loop

In [ ]:
best_loss = np.inf

qqdm_train = qqdm(range(num_epochs), desc=format_str('bold', 'Description'), dynamic_ncols=False)
for epoch in qqdm_train:
    tot_loss = list()

    for data in train_dataloader:
        # ===================loading=====================
        img = data.float().cuda()
        if model_type in ['fcn']:
            img = img.view(img.shape[0], -1)
        # ===================train autoencoder=====================
        model.train()
        output = model(img)
        if model_type in ['vae']:
            loss = loss_vae(output[0], img, output[1], output[2], criterion)
        else:
            loss = criterion(output, img)
        loss = loss.mean()
        tot_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # ===================save_best====================
    mean_loss = np.mean(tot_loss)
    if mean_loss < best_loss:
        best_loss = mean_loss
        torch.save(model, 'best_model_{}.pt'.format(model_type))
    # ===================log========================
    qqdm_train.set_infos({
        'epoch': f'{epoch + 1:.0f}/{num_epochs:.0f}',
        'loss':f'{np.mean(tot_loss):.5f}',
    })


# ===================save_last========================
torch.save(model, 'last_model_{}.pt'.format(model_type))


 Iters    Elapsed Time      Speed                                               
 0/100          -             -                                                 
Description   0.0% |                                       |                     Iters    Elapsed Time      Speed    epoch   loss                               
 1/100  00:00:03<00:06:01  0.27it/s  1/100  0.06257                             
Description   1.0% |                                       |                     Iters    Elapsed Time      Speed    epoch   loss                               
 2/100  00:00:06<00:05:33  0.29it/s  2/100  0.04450                             
Description   2.0% |                                       |                     Iters    Elapsed Time      Speed    epoch   loss                               
 3/100  00:00:09<00:05:22  0.30it/s  3/100  0.04269                             
Description   3.0% |█                                      |                     Iters    Elapsed Time      Spee

Description  20.0% |███████                                |                     Iters     Elapsed Time      Speed    epoch    loss                             
 21/100  00:01:06<00:04:11  0.31it/s  21/100  0.03860                           
Description  21.0% |████████                               |                     Iters     Elapsed Time      Speed    epoch    loss                             
 22/100  00:01:10<00:04:08  0.31it/s  22/100  0.03853                           
Description  22.0% |████████                               |                     Iters     Elapsed Time      Speed    epoch    loss                             
 23/100  00:01:13<00:04:05  0.31it/s  23/100  0.03847                           
Description  23.0% |████████                               |                     Iters     Elapsed Time      Speed    epoch    loss                             
 24/100  00:01:16<00:04:01  0.31it/s  24/100  0.03839                           
Description  24.0% |█████████   

Description  38.0% |██████████████                         |                     Iters     Elapsed Time      Speed    epoch    loss                             
 39/100  00:02:03<00:03:13  0.32it/s  39/100  0.03764                           
Description  39.0% |███████████████                        |                     Iters     Elapsed Time      Speed    epoch    loss                             
 40/100  00:02:06<00:03:10  0.32it/s  40/100  0.03760                           
Description  40.0% |███████████████                        |                     Iters     Elapsed Time      Speed    epoch    loss                             
 41/100  00:02:10<00:03:07  0.32it/s  41/100  0.03760                           
Description  41.0% |███████████████                        |                     Iters     Elapsed Time      Speed    epoch    loss                             
 42/100  00:02:13<00:03:03  0.32it/s  42/100  0.03750                           
Description  42.0% |████████████

Description  54.0% |█████████████████████                  |                     Iters     Elapsed Time      Speed    epoch    loss                             
 55/100  00:02:54<00:02:22  0.32it/s  55/100  0.03717                           
Description  55.0% |█████████████████████                  |                     Iters     Elapsed Time      Speed    epoch    loss                             
 56/100  00:02:57<00:02:19  0.32it/s  56/100  0.03717                           
Description  56.0% |█████████████████████                  |                     Iters     Elapsed Time      Speed    epoch    loss                             
 57/100  00:03:00<00:02:16  0.32it/s  57/100  0.03711                           
Description  57.0% |██████████████████████                 |                     Iters     Elapsed Time      Speed    epoch    loss                             
 58/100  00:03:03<00:02:13  0.32it/s  58/100  0.03707                           
Description  58.0% |████████████

# Inference
Model is loaded and generates its anomaly score predictions.

## Initialize
- dataloader
- model
- prediction file

In [ ]:
eval_batch_size = 200

# build testing dataloader
data = torch.tensor(test, dtype=torch.float32)
test_dataset = CustomTensorDataset(data)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=eval_batch_size, num_workers=1)
eval_loss = nn.MSELoss(reduction='none')

# load trained model
checkpoint_path = f'best_model_{model_type}.pt'
model = torch.load(checkpoint_path)
model.eval()

# prediction file 
out_file = 'prediction.csv'

In [ ]:
anomality = list()
with torch.no_grad():
  for i, data in enumerate(test_dataloader):
    img = data.float().cuda()
    if model_type in ['fcn']:
      img = img.view(img.shape[0], -1)
    output = model(img)
    if model_type in ['vae']:
      output = output[0]
    if model_type in ['fcn']:
        loss = eval_loss(output, img).sum(-1)
    else:
        loss = eval_loss(output, img).sum([1, 2, 3])
    anomality.append(loss)
anomality = torch.cat(anomality, axis=0)
anomality = torch.sqrt(anomality).reshape(len(test), 1).cpu().numpy()

df = pd.DataFrame(anomality, columns=['score'])
df.to_csv(out_file, index_label = 'ID')

# Gradescope

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# checkpoint_path = f'best_model_fcn_{seed}.pt'
# model = fcn_autoencoder()
# model = torch.load(checkpoint_path)
# model.eval()
# with torch.no_grad():
#     img = test_dataset[42].cuda()
#     img = (img+1)/2
#     image = np.transpose(img.cpu(), (1, 2, 0))
#     plt.imshow(image)
#     plt.show()
#     shape = img.shape
#     img = test_dataset[42].cuda().reshape([3*64*64])
#     output = (model(img) + 1) / 2 
#     output = output.reshape(shape).float().cuda()
#     output = np.transpose(output.cpu(), (1, 2, 0))
#     plt.imshow(output)
#     plt.show()